# LSSTCam pointing errors
# Looking at the encoder offsets.

Craig Lage - 02-Jul-25

In [ ]:
import os, sys
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.summit.utils.tmaUtils import TMAEventMaker

In [ ]:
client = makeEfdClient()
eventMaker = TMAEventMaker()

# First, get the tracking events for which Hi-Res data exists and categorize them.

In [ ]:
start_hires = Time("2025-06-30T00:00:00")
end_hires = Time("2025-06-30T02:00:00")

dayObs = 20250629
pdf = PdfPages(f"/home/c/cslage/u/MTMount/mount_plots/Offset_vs_AzEl{dayObs}.pdf")
fig = plt.figure(figsize=(8, 5))
events = eventMaker.getEvents(dayObs)
myEvents = [e for e in events if ((e.type.name=='TRACKING') and (e.duration > 29.0) and (e.begin > start_hires) and (e.end < end_hires))]
print(f"for {dayObs}, there were {len(myEvents)} tracks that met the criteria")

# This weeds out slews with next to zero motion.
goodEvents = []
for event in myEvents:
    az = getEfdData(
        client,
        "lsst.sal.MTMount.azimuth",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    el = getEfdData(
        client,
        "lsst.sal.MTMount.elevation",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    azValues = np.asarray(az["actualPosition"])
    elValues = np.asarray(el["actualPosition"])
    dAz = abs(azValues[0] - azValues[-1])
    dEl = abs(elValues[0] - elValues[-1])
    #print(event.seqNum, dAz, dEl)
    if (dAz > 0.01) and (dEl > 0.01):
        goodEvents.append(event)
print(f"for {dayObs}, there were {len(goodEvents)} tracks that met the criteria")
hiResEvents = {}
for event in goodEvents:
    seqNum = event.seqNum
    hiResEvents[seqNum] = [event.begin, event.end]

filename = "/home/c/cslage/u/MTMount/hi_res_data/HiResEvents.pkl"
with open(filename, 'wb') as f:
    pkl.dump(hiResEvents, f)


# Now we have parsed the HiRes data into individual pickle files and we can get them one at a time
## This was done on my laptop, since the file from Julen was to large to upload.

In [ ]:
dayObs = 20250629
pdf = PdfPages(f"/home/c/cslage/u/MTMount/mount_plots/Offset_vs_AzEl_HiRes_Trimmed_{dayObs}.pdf")
fig = plt.figure(figsize=(8, 5))
azs = []
els = []
azOffsets = []
elOffsets = []
seqNums = []

for seqNum in hiResEvents.keys():
    infile = f"/home/c/cslage/u/MTMount/hi_res_data/hiResData_Trimmed_{seqNum}.pkl"
    with open(infile, "rb") as f:
        [az, el] = pkl.load(f)
        if len(az) > 1E6:
            skip = 100
        else:
            skip = 10
    seqNums.append(seqNum)
    axs = []
    axs.append(fig.add_subplot(1,2,1))
    axs.append(fig.add_subplot(1,2,2))
    plt.subplots_adjust(wspace = 0.5)
    
    azValues = np.asarray(az["actualPosition"])
    azs.append(azValues[0])
    azValTimes = np.asarray(az["actualPositionTimestamp"])
    azEncInterp = np.zeros_like(azValues)
    for ii in range(4):
        azEncValsN = np.asarray(az[f'azimuthEncoderPosition{ii}'])
        azEncTimesN = np.asarray(az[f'azimuthEncoderPositionTimestamp{ii}'])
        azEncInterpN = np.interp(azValTimes, azEncTimesN, azEncValsN)
        azEncInterp += azEncInterpN
    azEncInterp /= 4.0      
    
    elValues = np.asarray(el["actualPosition"])
    els.append(elValues[0])
    elValTimes = np.asarray(el["actualPositionTimestamp"])
    elEncInterp = np.zeros_like(elValues)
    for ii in range(4):
        elEncValsN = np.asarray(el[f'elevationEncoderPosition{ii}'])
        elEncTimesN = np.asarray(el[f'elevationEncoderPositionTimestamp{ii}'])
        elEncInterpN = np.interp(elValTimes, elEncTimesN, elEncValsN)
        elEncInterp += elEncInterpN
    elEncInterp /= 4.0      

    axs[0].set_title(f"{dayObs}, Tracking event {seqNum}")
    axs[0].scatter(azValues[::skip], azEncInterp[::skip])
    azFit = np.polyfit(azValues, azEncInterp, 1)
    #print(fit)
    axs[0].text(0.1, 0.8, f"Slope = {azFit[0]:.6f}", transform=axs[0].transAxes)
    axs[0].text(0.1, 0.7, f"Offset = {(azFit[1]*3600):.2f} arcseconds", transform=axs[0].transAxes)
    azOffsets.append((azFit[1]*3600))
    axs[0].set_xlabel("Azimuth (degrees)")
    axs[0].set_ylabel("Azimuth Encoder average(degrees)")
    axs[1].scatter(elValues[::skip], elEncInterp[::skip])
    elFit = np.polyfit(elValues, elEncInterp, 1)
    #print(fit)
    axs[1].set_title(f"{dayObs}, Tracking event {seqNum}")
    axs[1].text(0.1, 0.8, f"Slope = {elFit[0]:.6f}", transform=axs[1].transAxes)
    axs[1].text(0.1, 0.7, f"Offset = {(elFit[1]*3600):.2f} arcseconds", transform=axs[1].transAxes)
    elOffsets.append((elFit[1]*3600))
    axs[1].set_xlabel("Elevation (degrees)")
    axs[1].set_ylabel("Elevation Encoder average(degrees)")
    pdf.savefig(fig)
    fig.clf()
    print(f"Finished seqNum {seqNum}")
pdf.close()  

fig = plt.figure(figsize=(8,5))
plt.subplots_adjust(wspace=0.5)
plt.suptitle(f"Encoder offsets {dayObs}")
plt.subplot(1,2,1)
plt.scatter(azs, azOffsets)
plt.ylim(-10,10)
plt.xlabel("Azimuth(degrees)")
plt.ylabel("Azimuth offset (arcseconds)")
plt.subplot(1,2,2)
plt.scatter(els, elOffsets)
plt.ylim(-10,10)
plt.xlabel("Elevation(degrees)")
plt.ylabel("Elevation offset (arcseconds)")
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Encoder_Offset_Summary_HiRes_Trimmed_{dayObs}.png")

In [ ]:
dayObs = 20250629
pdf = PdfPages(f"/home/c/cslage/u/MTMount/mount_plots/Residuals_HiRes_Trimmed_{dayObs}.pdf")
fig = plt.figure(figsize=(8, 5))
azs = []
els = []
azOffsets = []
elOffsets = []
seqNums = []

for seqNum in hiResEvents.keys():
    infile = f"/home/c/cslage/u/MTMount/hi_res_data/hiResData_{seqNum}.pkl"
    with open(infile, "rb") as f:
        [az, el] = pkl.load(f)
        if len(az) > 1E6:
            skip = 100
        else:
            skip = 10
    seqNums.append(seqNum)
    axs = []
    axs.append(fig.add_subplot(1,2,1))
    axs.append(fig.add_subplot(1,2,2))
    plt.subplots_adjust(wspace = 0.5)
    
    azValues = np.asarray(az["actualPosition"])
    azs.append(azValues[0])
    azValTimes = np.asarray(az["actualPositionTimestamp"])
    azEncInterp = np.zeros_like(azValues)
    for ii in range(4):
        azEncValsN = np.asarray(az[f'azimuthEncoderPosition{ii}'])
        azEncTimesN = np.asarray(az[f'azimuthEncoderPositionTimestamp{ii}'])
        azEncInterpN = np.interp(azValTimes, azEncTimesN, azEncValsN)
        azEncInterp += azEncInterpN
    azEncInterp /= 4.0      
    
    elValues = np.asarray(el["actualPosition"])
    els.append(elValues[0])
    elValTimes = np.asarray(el["actualPositionTimestamp"])
    elEncInterp = np.zeros_like(elValues)
    for ii in range(4):
        elEncValsN = np.asarray(el[f'elevationEncoderPosition{ii}'])
        elEncTimesN = np.asarray(el[f'elevationEncoderPositionTimestamp{ii}'])
        elEncInterpN = np.interp(elValTimes, elEncTimesN, elEncValsN)
        elEncInterp += elEncInterpN
    elEncInterp /= 4.0      

    axs[0].set_title(f"{dayObs}, Tracking event {seqNum}")
    azFit = np.polyfit(azValues, azEncInterp, 1)
    azFitVals = np.polyval(azFit, azValues)
    axs[0].scatter(azValues[::skip], (azEncInterp[::skip] - azFitVals[::skip]) * 3600.0)
    #print(fit)
    axs[0].text(0.1, 0.8, f"Slope = {azFit[0]:.6f}", transform=axs[0].transAxes)
    axs[0].text(0.1, 0.7, f"Offset = {(azFit[1]*3600):.2f} arcseconds", transform=axs[0].transAxes)
    azOffsets.append((azFit[1]*3600))
    axs[0].set_xlabel("Azimuth (degrees)")
    axs[0].set_ylabel("Azimuth Encoder residuals(arcsec)")
    axs[0].set_ylim(-1, 1)
    
    #print(fit)
    axs[1].set_title(f"{dayObs}, Tracking event {seqNum}")
    elFit = np.polyfit(elValues, elEncInterp, 1)
    elFitVals = np.polyval(elFit, elValues)
    axs[1].scatter(elValues[::skip], (elEncInterp[::skip] - elFitVals[::skip]) * 3600.0)
    axs[1].text(0.1, 0.8, f"Slope = {elFit[0]:.6f}", transform=axs[1].transAxes)
    axs[1].text(0.1, 0.7, f"Offset = {(elFit[1]*3600):.2f} arcseconds", transform=axs[1].transAxes)
    elOffsets.append((elFit[1]*3600))
    axs[1].set_xlabel("Elevation (degrees)")
    axs[1].set_ylabel("Elevation Encoder residuals(arcsec)")
    axs[1].set_ylim(-1, 1)
    pdf.savefig(fig)
    fig.clf()
    print(f"Finished seqNum {seqNum}")
pdf.close()  
